## EDA notebook

In [1]:
import pandas as pd
import sqlite3

import data_preparation as dp

df_p = pd.read_csv('../data/tn.movie_budgets.csv.gz')
df_g = pd.read_csv('../data/tmdb.movies.csv.gz')

con = sqlite3.connect("../data/im.db")
df_r = pd.read_sql("""SELECT * FROM movie_basics;""", con)

In [2]:
set_year = 2000
set_profit = 10

In [3]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [4]:
df_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [5]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


## Profit Data

Budget amount and worldwide_gross amount are converted to integers. \
Profit = (budget - worldwide_gross) / 1000000

In [6]:
len(df_p.movie)

5782

In [7]:
df_p['date'] = pd.to_datetime(df_p.release_date)
df_p['date'].describe(datetime_is_numeric=True)

count                             5782
mean     2004-07-06 05:20:31.546177792
min                1915-02-08 00:00:00
25%                2000-04-22 18:00:00
50%                2007-03-02 00:00:00
75%                2012-12-25 00:00:00
max                2020-12-31 00:00:00
Name: date, dtype: object

In [8]:
df_p = dp.profit(df_p, set_year, set_profit)

In [9]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2102 entries, 0 to 4018
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              2102 non-null   object        
 1   production_budget  2102 non-null   object        
 2   worldwide_gross    2102 non-null   object        
 3   date               2102 non-null   datetime64[ns]
 4   year               2102 non-null   int64         
 5   budget             2102 non-null   int32         
 6   w_gross            2102 non-null   float64       
 7   profit             2102 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(1), object(3)
memory usage: 139.6+ KB


In [10]:
df_p.profit.describe()

count    2102.000000
mean      136.891327
std       195.696018
min        10.010000
25%        28.877500
50%        67.165000
75%       151.017500
max      2351.350000
Name: profit, dtype: float64

In [11]:
df_p.budget.describe()

count    2.102000e+03
mean     5.292571e+07
std      5.418069e+07
min      6.500000e+04
25%      1.512500e+07
50%      3.500000e+07
75%      7.192074e+07
max      4.250000e+08
Name: budget, dtype: float64

## Genre Data

In [12]:
len(df_g)

26517

In [13]:
type(df_g.genre_ids[0])

str

In [14]:
# count of each genre before inner merge
Genre = dp.genre_count(df_g)
Genre

,Genre_name,Genre_count
18,Western,205
17,War,330
9,History,622
15,TV_Movie,1084
8,Fantasy,1139
12,Mystery,1237
11,Music,1267
1,Adventure,1400
2,Animation,1486
4,Crime,1515


In [15]:
df_g = dp.genre(df_g)

In [16]:
df_g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25494 entries, 0 to 26516
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genre_ids     25494 non-null  object
 1   release_date  25494 non-null  object
 2   title         25494 non-null  object
dtypes: object(3)
memory usage: 796.7+ KB


## Runtime data

In [17]:
len(df_r)

146144

In [18]:
df_r.runtime_minutes.describe()

count    114405.000000
mean         86.187247
std         166.360590
min           1.000000
25%          70.000000
50%          87.000000
75%          99.000000
max       51420.000000
Name: runtime_minutes, dtype: float64

In [19]:
df_r = dp.runtime(con)

In [20]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114405 entries, 0 to 114404
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         114405 non-null  object 
 1   primary_title    114405 non-null  object 
 2   start_year       114405 non-null  int64  
 3   runtime_minutes  114405 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ MB


## Merge
Movie titles are lower-cased and spaces and punctuations are removed before merging.

In [21]:
df_p.movie.head()

0                                   Avatar
6                   Avengers: Infinity War
5     Star Wars Ep. VII: The Force Awakens
33                          Jurassic World
66                               Furious 7
Name: movie, dtype: object

In [22]:
df_g.title.head()

0    Harry Potter and the Deathly Hallows: Part 1
1                        How to Train Your Dragon
2                                      Iron Man 2
3                                       Toy Story
4                                       Inception
Name: title, dtype: object

In [23]:
df_r.primary_title.head()

0                          Sunghursh
1    One Day Before the Rainy Season
2         The Other Side of the Wind
3           The Wandering Soap Opera
4                        A Thin Life
Name: primary_title, dtype: object

In [24]:
df_p.movie = dp.movie_name_clean(df_p.movie)
df_g.title = dp.movie_name_clean(df_g.title)
df_r.primary_title = dp.movie_name_clean(df_r.primary_title)

In [25]:
df_p.movie.head()

0                           avatar
6              avengersinfinitywar
5     starwarsepviitheforceawakens
33                   jurassicworld
66                        furious7
Name: movie, dtype: object

In [26]:
df_g.title.head()

0    harrypotterandthedeathlyhallowspart1
1                    howtotrainyourdragon
2                                ironman2
3                                toystory
4                               inception
Name: title, dtype: object

In [27]:
df_r.primary_title.head()

0                     sunghursh
1    onedaybeforetherainyseason
2         theothersideofthewind
3         thewanderingsoapopera
4                     athinlife
Name: primary_title, dtype: object

In [28]:
df_genre = df_p.merge(df_g, how = 'inner', left_on = 'movie', right_on = 'title')
df_runtime = df_p.merge(df_r, how = 'inner', left_on = 'movie', right_on = 'primary_title')

In [29]:
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 0 to 1120
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              1121 non-null   object        
 1   production_budget  1121 non-null   object        
 2   worldwide_gross    1121 non-null   object        
 3   date               1121 non-null   datetime64[ns]
 4   year               1121 non-null   int64         
 5   budget             1121 non-null   int32         
 6   w_gross            1121 non-null   float64       
 7   profit             1121 non-null   float64       
 8   genre_ids          1121 non-null   object        
 9   release_date       1121 non-null   object        
 10  title              1121 non-null   object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(1), object(6)
memory usage: 100.7+ KB


In [30]:
# count of each genre after inner merge
Genre = dp.genre_count(df_genre)
Genre

,Genre_name,Genre_count
15,TV_Movie,2
18,Western,9
5,Documentary,21
17,War,24
11,Music,25
9,History,37
12,Mystery,81
2,Animation,102
4,Crime,125
13,Romance,140


In [31]:
df_runtime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 0 to 1511
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie              1512 non-null   object        
 1   production_budget  1512 non-null   object        
 2   worldwide_gross    1512 non-null   object        
 3   date               1512 non-null   datetime64[ns]
 4   year               1512 non-null   int64         
 5   budget             1512 non-null   int32         
 6   w_gross            1512 non-null   float64       
 7   profit             1512 non-null   float64       
 8   movie_id           1512 non-null   object        
 9   primary_title      1512 non-null   object        
 10  start_year         1512 non-null   int64         
 11  runtime_minutes    1512 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(5)
memory usage: 147.7+ KB
